## 대용량 json 파일 구조 확인하기

In [ ]:
%%cmd
head C:/Users/default.DESKTOP-FVT3076/Desktop/data/data.json
#안되니까 git bash로 확인
#한줄 데이터라 byte 단위로 json 구조 확인
head $ head data.json -c 1000

In [ ]:
!jupyter notebook --generate-config

## 데이터 전처리

In [ ]:
#데이터가 너무 커서 블록단위로 메모리에 올리는 ijson 사용
import pandas as pd
import ijson

f = open(f"./data.json", encoding="UTF-8")
objects = ijson.items(f, 'item')
origin = pd.DataFrame(objects)
# df = pd.read_csv('./data/성북구_음식점.csv', sep=',', encoding='CP949')

# df = df[['업소명', '소재지도로명', '업태명', '주된음식', '행정동명', '소재지전화번호']]
# df.columns = ['name', 'address', 'cate1', 'cate2', 'dong', 'phone']
# df = df.drop_duplicates(['name'], keep='first')
# # 네이버 지도 검색창에 [~동 @@식당]으로 검색해 정확도를 높여야 합니다. 검색어를 미리 설정해줍시다.
# df['cate_mix'] = df['cate1'] + df['cate2']
print(origin)

In [ ]:
for i in range(0,df.shape[0]):
    if df.iloc[i]['area'] is None or df.iloc[i,-2] != df.iloc[i,-2]: #nan 찾는 방법
        df.iloc[i,-2] = df.iloc[i]['name']

In [ ]:
# df.at[90,'google_keyword'] = df.iloc[90]['name']
# df.at[90,'google_keyword']
origin.columns

## 데이터 셋 분할

In [ ]:
# 중복 키워드 삭제
origin = origin.drop_duplicates(['google_keyword'], keep='first') # 4371 사라짐
print(origin['google_keyword'].value_counts())
print(origin.shape)

In [ ]:
#필요한 칼럼
origin = origin[['name','area', 'address','latitude','longitude','google_keyword']]

In [ ]:
#구글맵용 가게정보 data 5등분 하기
import os
from datetime import datetime

start = 0
for i in range(1, 6):
    end = (df1.shape[0]//5) * i
    temp = df1[start:end]
    file = 'GoogleMap용_가게정보_part_'+str(i) +'.csv'
    temp.to_csv(file, encoding='utf-8')
    print(file)
    start = end


In [ ]:
#scweet용 가게정보 data 5등분 하기
import os
from datetime import datetime

df = df.drop_duplicates(['name'], keep='first') # 4371 사라짐

start = 0
for i in range(1, 6):
    end = (df.shape[0]//5) * i
    df1 = df['name'][start:end]
    file = 'scweet용_가게정보_' + str(end) + '행까지.csv'
    df1.to_csv(file, encoding='utf-8')
    print(file)
    start = end


## ipynb to py

In [ ]:
!pip install nbconvert

In [ ]:
!jupyter nbconvert --to script naver_search_api.ipynb

## pandas list column

In [ ]:
import pandas as pd

df = pd.read_csv('../outputs/BERT용 data/BERT_0~20000행_2021-10-03.csv', sep=',', encoding='utf-8')
df

In [ ]:
df = df.loc[~df['google_review_txt'].isnull()]

In [ ]:
from ast import literal_eval

df['google_review_txt'] = df['google_review_txt'].apply(literal_eval)
df['google_review_txt'].iloc[0]

## 크롤링 결과 합치기 & 리뷰없는 맛집은 ES에도 넣지 않는다
### 두 데이터프레임에서 같은 keyword 찾기
* BERT용에서 리뷰없는거 지우고 그 keyword로 origin 데이터에서 contains로 찾아낸다
* contains 비교용 리스트 만들때 특수문자 제거해야함

In [ ]:
import pandas as pd
import ijson

f = open(f"./data.json", encoding="UTF-8")
objects = ijson.items(f, 'item')
origin = pd.DataFrame(objects)

origin["name"] = origin["name"].str.replace('&amp;', '') # str로 Series를 하나하나 파싱해서 지워야함
origin['google_keyword'] = origin['area'] + origin['name']
origin = origin.loc[~origin['google_keyword'].isnull()]
origin = origin.drop_duplicates(['google_keyword'], keep='first') # 4371 사라짐
origin = origin[['name','area', 'address','latitude','longitude','google_keyword']]

In [ ]:
df = pd.read_csv('./BERT_30000~40000행_2021-10-06.csv', sep=',', encoding='utf-8')
df = df.drop(['Unnamed: 0'], axis=1)
df["google_keyword"] = df["google_keyword"].str.replace('&amp;', '') # str로 Series를 하나하나 파싱해서 지워야함
df["google_keyword"] = df["google_keyword"].str.replace(pat=r'[^\w]', repl=r'', regex=True)
df = df.loc[~df['google_review_txt'].isnull()]
df

In [ ]:
lis = df['google_keyword'].tolist()
listemp = '|'.join(lis)
result1 = origin[origin['google_keyword'].str.contains(listemp)]
result1

In [ ]:
file = 'ES_part_17.csv'
result1.to_csv(file, encoding='utf-8')
print(file)

In [ ]:
idx = df[df['google_keyword'].str.contains("?")].index
print(idx)
# df = df.drop(idx)
# df[df['google_keyword'].str.contains("롯데월드몰Wetzel's ")]

### csv 파일 합치기

In [ ]:
import pandas as pd
import glob
import os

input_file = r'C:\Users\default.DESKTOP-FVT3076\Desktop\저장 데이터\null처리후' # csv파일들이 있는 디렉토리 위치
output_file = r'C:\Users\default.DESKTOP-FVT3076\Desktop\저장 데이터\null처리후\ES_total.csv' # 병합하고 저장하려는 파일명

allFile_list = glob.glob(os.path.join(input_file, 'ES_part_*')) # glob함수로 sales_로 시작하는 파일들을 모은다
# print(allFile_list)
allData = [] # 읽어 들인 csv파일 내용을 저장할 빈 리스트를 하나 만든다
for file in allFile_list:
    df = pd.read_csv(file) # for구문으로 csv파일들을 읽어 들인다
    allData.append(df) # 빈 리스트에 읽어 들인 내용을 추가한다
print(allData)
dataCombine = pd.concat(allData, axis=0, ignore_index=True) # concat함수를 이용해서 리스트의 내용을 병합
# axis=0은 수직으로 병합함. axis=1은 수평. ignore_index=True는 인데스 값이 기존 순서를 무시하고 순서대로 정렬되도록 한다.
dataCombine.to_csv(output_file, index=False) # to_csv함수로 저장한다. 인데스를 빼려면 False로 설정

In [ ]:
dataCombine

## send to ElasticSearch

In [ ]:
pip install elasticsearch

In [ ]:
from elasticsearch import Elasticsearch
import pandas as pd

df = pd.read_csv('../outputs/ES용 data/ES_0~20000행_2021-10-03.csv', sep=',', encoding='utf-8')

In [ ]:
df["name"] = df["name"].str.replace('&amp;', '') # str로 Series를 하나하나 파싱해서 지워야함
df[df['name'].str.contains("&amp;")] # check해서 아무것도 안나오면 정상

In [ ]:
def setStore(i, store):
    
    try:
        es = Elasticsearch(
            hosts=[{'host':'3.34.51.207', 'port':'9200'}],
            http_auth=('elastic','msg!234'))
        es.index(index="msg", document=store)
    except:
        print(store)

for d in df.values:
#     print(d)
    store = {
        "name" : d[1],
        "area" : d[2],
        "address" : d[3],
        "latitude" : d[4],
        "longitude" : d[5]
    }  
    
    setStore(d[0],store)